In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Пути к файлам
btc_path = "../data/btc_futures.csv"
xrp_path = "../data/xrp_futures.csv"

# Загрузка
btc_df = pd.read_csv(btc_path)
xrp_df = pd.read_csv(xrp_path)

# Посмотрим на структуру
print("BTC columns:", btc_df.columns)
print("XRP columns:", xrp_df.columns)

# Построим графики цены или объема
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(btc_df["price"] if "price" in btc_df.columns else btc_df.iloc[:, 1])
plt.title("BTC Futures")
plt.xlabel("Time")
plt.ylabel("Price or Feature")

plt.subplot(1, 2, 2)
plt.plot(xrp_df["price"] if "price" in xrp_df.columns else xrp_df.iloc[:, 1])
plt.title("XRP Futures")
plt.xlabel("Time")
plt.ylabel("Price or Feature")

plt.tight_layout()
plt.show()
